In [1]:
import psycopg2

# create db connnection
def create_connection():
    try:
        connnection = psycopg2.connect(
            dbname="MusicStreamingDB",
            user="postgres",
            password="postgres",
            host="localhost"
        )
        return connnection
    except Exception as e:
        print('Error connecting to the db:', e)
        return None

    connection.create_connection()
    cursor = connection.cursor()

In [2]:
# Function to execute SQL query and print the result
def execute_sql(query, fetch_result=False):
    connection = create_connection()
    if connection:
        try:
            cursor = connection.cursor()
            cursor.execute(query)
            
            if fetch_result:
                result = cursor.fetchall()  # Fetch all results for SELECT queries
                return result  # Return result to be printed by the caller
            
            connection.commit()  # Commit changes 
            print("Query executed successfully.")
        except Exception as e:
            print("Error executing SQL query:", e)
        finally:
            cursor.close()
            connection.close()

# Function to run a query and print results for SELECT queries
def execute_and_print_query(query):
    result = execute_sql(query, fetch_result=True)
    if result:
        for row in result:
            print(row)
    else:
        print("No data returned.")

In [6]:
'''
    #8 An example of a view that has a hard-coded criteria, by which the content of the view may change upon changing the hard-coded value
'''
# This query will display the spotify song's with a popularity score above 80
QUERY = """
CREATE OR REPLACE VIEW HighPopularitySpotifySongs AS 
SELECT S.title, S.artist, S.genre, SS.spotify_popularity_score AS popularity_scr
FROM songs S
JOIN spotifySongs SS ON S.song_id = SS.song_id
WHERE SS.spotify_popularity_score > 80;
"""
execute_and_print_query(QUERY)

Error executing SQL query: no results to fetch
No data returned.


In [4]:
'''
    #9 Two queries that demonstrate the overlap and covering constraints.
'''
# Overlap:
QUERY1 = """
SELECT S.title FROM Songs S
JOIN SpotifySongs SS ON S.song_id = SS.song_id
JOIN LastfmSongs LFMS ON S.song_id = LFMS.song_id;
"""

# Covering:
QUERY2= """ 
SELECT S.title 
FROM Songs S 
WHERE NOT EXISTS (
    SELECT 1 FROM SpotifySongs AS SS 
    WHERE SS.song_id = S.song_id
)
AND NOT EXISTS (
    SELECT 1 FROM LastfmSongs AS LFMS
    WHERE LFMS.song_id = S.song_id
);
"""
execute_and_print_query(QUERY1)
execute_and_print_query(QUERY2)

No data returned.
No data returned.


In [5]:
'''
    #10 Two implementations of the division operator using 
         a) a regular nested query using NOT IN 
         b) a correlated nested query using NOT EXISTS and EXCEPT 
'''
# Both questies ensure that a song exists in both Spotify and Sound Cloud
# Using NOT IN:
QUERY1 = """
SELECT S.title FROM Songs S
WHERE S.song_id NOT IN (
    SELECT SS.song_id FROM Songs S
    LEFT JOIN SpotifySongs SS ON S.song_id = SS.song_id
    WHERE SS.song_id IS NULL
)
AND S.song_id NOT IN (
    SELECT LFMS.song_id FROM Songs S 
    LEFT JOIN LastfmSongs LFMS ON S.song_id = LFMS.song_id
    WHERE LFMS.song_id IS NULL
);
"""

# Using NOT EXISTS and EXCEPT:
QUERY2 = """ 
SELECT S.title FROM Songs S
WHERE NOT EXISTS (
    SELECT 1 FROM (
        SELECT song_id FROM SpotifySongs
        EXCEPT 
        SELECT song_id FROM LastfmSongs
    ) AS SpotifyButNotInSoundCloud
    WHERE S.song_id = SpotifyButNotInSoundCloud.song_id
);
"""
execute_and_print_query(QUERY1)
execute_and_print_query(QUERY2)

No data returned.
('Rah Tah Tah',)
('NOID',)
('ST. CHROMA (feat. Daniel Caesar)',)
('Hey Jane',)
('I Killed You',)
('Good Luck, Babe!',)
('Judge Judy',)
('BIRDS OF A FEATHER',)
('Darling, I (feat. Teezo Touchdown)',)
('STICKY (feat. GloRilla, Sexyy Red & Lil Wayne)',)
('APT.',)
('TOMORROW',)
('See You Again (feat. Kali Uchis)',)
('tAsTe',)
('espresso',)
('Die With A Smile',)
('Like Him (feat. Lola Young)',)
('ST. CHROMA',)
('I Hope You Find Your Way Home',)
('Thought I Was Dead (feat. ScHoolboy Q & Santigold)',)


In [7]:
# 4 Queries to demonstrate various join types on the same tables: inner vs. outer
# (left and right) vs. full join. Use of null values in the database to show the differences
# is required.

# An INNER JOIN returns records that have matching values in both tables.

QUERY1 = """
SELECT
    S.title,
    S.artist,
    SS.spotify_popularity_score,
    LF.lastfm_stream_count
FROM
    Songs S
INNER JOIN
    SpotifySongs SS ON S.song_id = SS.song_id
INNER JOIN
    LastfmSongs LF ON S.song_id = LF.song_id;
"""

# A LEFT OUTER JOIN returns all records from the left table (Songs), and the matched records from the right tables (SpotifySongs and LastfmSongs). If there is no match, the result is NULL on the side of the right table.

QUERY2 = """
SELECT
    S.title,
    S.artist,
    SS.spotify_popularity_score,
    LF.lastfm_stream_count
FROM
    Songs S
RIGHT OUTER JOIN
    SpotifySongs SS ON S.song_id = SS.song_id
RIGHT OUTER JOIN
    LastfmSongs LF ON S.song_id = LF.song_id;
"""

# A RIGHT OUTER JOIN returns all records from the right table and the matched records from the left table. If there is no match, the result is NULL on the side of the left table.

QUERY3 = """
SELECT
    S.title,
    S.artist,
    SS.spotify_popularity_score,
    LF.lastfm_stream_count
FROM
    Songs S
FULL OUTER JOIN
    SpotifySongs SS ON S.song_id = SS.song_id
FULL OUTER JOIN
    LastfmSongs LF ON S.song_id = LF.song_id;
"""

# A FULL OUTER JOIN returns all records when there is a match in either left or right table. If there is no match, the result is NULL on the side that doesn't have a match.

QUERY4 = """

SELECT
    S.title,
    S.artist,
    SS.spotify_popularity_score,
    LF.lastfm_stream_count
FROM
    Songs S
FULL OUTER JOIN
    SpotifySongs SS ON S.song_id = SS.song_id
FULL OUTER JOIN
    LastfmSongs LF ON S.song_id = LF.song_id;
"""
execute_and_print_query(QUERY1)
execute_and_print_query(QUERY2)
execute_and_print_query(QUERY3)
execute_and_print_query(QUERY4)

No data returned.
(None, None, None, 2418673)
(None, None, None, 32420396)
(None, None, None, 2418577)
(None, None, None, 1758728)
(None, None, None, 36731199)
(None, None, None, 8808339)
(None, None, None, 13438442)
(None, None, None, 2283534)
(None, None, None, 4090687)
(None, None, None, 3333807)
(None, None, None, 2202395)
(None, None, None, 2453343)
(None, None, None, 1287836)
(None, None, None, 28861561)
(None, None, None, 2918722)
(None, None, None, 2994619)
(None, None, None, 12453665)
(None, None, None, 3876399)
(None, None, None, 4763874)
(None, None, None, 33830018)
('Die With A Smile', 'Lady Gaga, Bruno Mars', 100, None)
('APT.', 'ROSÉ, Bruno Mars', 98, None)
('BIRDS OF A FEATHER', 'Billie Eilish', 97, None)
('That’s So True', 'Gracie Abrams', 93, None)
('Who', 'Jimin', 92, None)
('Running Wild', 'Jin', 71, None)
('Sailor Song', 'Gigi Perez', 93, None)
('Taste', 'Sabrina Carpenter', 93, None)
('Tu Boda', 'Oscar Maydon, Fuerza Regida', 93, None)
('WILDFLOWER', 'Billie Eilish

In [9]:
# 6 A couple of examples to demonstrate correlated queries.

# Listing Songs with More Last.fm Streams Than Any Other Song by the Same Artist
QUERY1 = """
SELECT
    S.title,
    S.artist,
    LF.lastfm_stream_count
FROM
    Songs S
JOIN
    LastfmSongs LF ON S.song_id = LF.song_id
WHERE
    LF.lastfm_stream_count > ALL (
        SELECT
            LF2.lastfm_stream_count
        FROM
            Songs S2
        JOIN
            LastfmSongs LF2 ON S2.song_id = LF2.song_id
        WHERE
            S2.artist = S.artist
            AND S2.song_id <> S.song_id
    );
"""

# One example per set operations: intersect, union, and difference vs. their equivalences
# without using set operations.


execute_and_print_query(QUERY1)




('Good Luck, Babe!', 'Chappell Roan', 32420396)
('BIRDS OF A FEATHER', 'Billie Eilish', 28861561)
('APT.', 'Rosé', 8808339)
('See You Again (feat. Kali Uchis)', 'Tyler, the Creator', 36731199)
('espresso', 'Sabrina Carpenter', 33830018)
('Die With A Smile', 'Lady Gaga', 12453665)
